In [1]:
%pip install requests
%pip install rsa


  Obtaining dependency information for requests from https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd6965bf26b47bc0bf44/requests-2.31.0-py3-none-any.whl.metadata
  Obtaining dependency information for charset-normalizer<4,>=2 from https://files.pythonhosted.org/packages/b6/7c/8debebb4f90174074b827c63242c23851bdf00a532489fba57fef3416e40/charset_normalizer-3.3.2-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for idna<4,>=2.5 from https://files.pythonhosted.org/packages/e5/3e/741d8c82801c347547f8a2a06aa57dbb1992be9e948df2ea0eda2c8b79e8/idna-3.7-py3-none-any.whl.metadata
  Obtaining dependency information for urllib3<3,>=1.21.1 from https://files.pythonhosted.org/packages/a2/73/a68704750a7679d0b6d3ad7aa8d4da8e14e151ae82e6fee774e6e0d05ec8/urllib3-2.2.1-py3-none-any.whl.metadata
  Obtaining dependency information for certifi>=2017.4.17 from https://files.pythonhosted.org/packages/ba/06/a07f096c664aeb9f01624f858c3add0a4e913d6c962


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for rsa from https://files.pythonhosted.org/packages/49/97/fa78e3d2f65c02c8e1268b9aba606569fe97f6c8f7c2d74394553347c145/rsa-4.9-py3-none-any.whl.metadata
  Obtaining dependency information for pyasn1>=0.1.3 from https://files.pythonhosted.org/packages/23/7e/5f50d07d5e70a2addbccd90ac2950f81d1edd0783630651d9268d7f1db49/pyasn1-0.6.0-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/85.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/85.3 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/85.3 kB ? eta -:--:--
   -------------- ------------------------- 30.7/85.3 kB 325.1 kB/s eta 0:00:01
   ------------------- -------------------- 41.0/85.3 kB 245.8 kB/s eta 0:00:01
   ---------------------------- ----------- 61.4/85.3 kB 297.7 kB/s eta 0:00:01
   -------------------------------------- - 81.9/85.3 kB 327.3 kB/s eta 0:00:01
   -------------------------------------- - 81.9/85.3 kB 327.3 kB/s eta 


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import requests
import rsa
import json
import binascii
import json
import random

G_BaseURL = "http://192.168.80.147/TESTwebservices_SWC"
#G_BaseURL = "http://78.158.168.230/TESTwebservices_SWC"
G_UserName = "admin"
G_PassWord = "admin"


def hex_string_to_bytes(hex_string):
    return binascii.unhexlify(hex_string)
def bytes_to_hex_string(byte_array):
    return binascii.hexlify(byte_array).decode()
def login(user_name=G_UserName, password=G_PassWord):
    url = G_BaseURL + "/Services/Framework/AuthenticationService.svc"
    session_url = url + "/session"
    login_url = url + "/login"
    response = requests.get(session_url)
    if response.status_code != 200:
        raise Exception("GET /session {}".format(response.status_code))
    session = json.loads(response.text)
    m = hex_string_to_bytes(session["rsa"]["M"])
    e = hex_string_to_bytes(session["rsa"]["E"])
    rsa_key = rsa.PublicKey(
        int.from_bytes(m, byteorder="big"), int.from_bytes(e, byteorder="big")
    )
    session_plus_password = session["id"] + "**" + password
    encrypted_password = rsa.encrypt(session_plus_password.encode(), rsa_key)
    headers = {"content-Type": "application/json"}
    data = {
        "sessionId": session["id"],
        "username": user_name,
        "password": bytes_to_hex_string(encrypted_password),
    }
    response = requests.post(login_url, headers=headers, data=json.dumps(data))
    if response.status_code != 200:
        raise Exception("POST /login {}".format(response.status_code))
    # session = response.headers["Set-Cookie"].split(",")[2].split(";")[0].strip()
    session = response.headers["Set-Cookie"].split(",")[1].split(";")[0].strip()
    return session


session = login()
print(session)

sg-auth-TESTwebservices_SWC=781fc90a-83cf-4465-bed4-4f03f78467fc


In [5]:
"""
    get all  GetRegionalDivisionList
"""



def GetRegionalDivisionList(session):
    url = (
        G_BaseURL
        + "/General/AddressManagement/Services/AddressManagementWebService.svc/GetRegionalDivisionList"
    )
    headers = {"Cookie": session, "content-Type": "application/json"}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        raise Exception("GET /GetAllCostCenters {}".format(response.status_code))
    return response.json()


# Use the session_id and auth_cookie from the login function
# cost_centers = get_all_cost_centers(session)
# print(cost_centers)
# json_str = json.dumps(your_list)
# parsed = json.loads(cost_centers)

# Pretty print the parsed data
# print(json.dumps(parsed, indent=4))

print(
    json.dumps(
        json.loads(json.dumps(GetRegionalDivisionList(session))),
        indent=4,
        ensure_ascii=False,
    )
)
# """

[
    {
        "Code": "0098",
        "Name": "ایران",
        "ParentID": null,
        "RegionalDivisionID": 1,
        "Type": 1
    },
    {
        "Code": null,
        "Name": "استان تهران",
        "ParentID": 1,
        "RegionalDivisionID": 2,
        "Type": 2
    },
    {
        "Code": "021",
        "Name": "تهران",
        "ParentID": 2,
        "RegionalDivisionID": 3,
        "Type": 3
    },
    {
        "Code": null,
        "Name": "شمیرانات",
        "ParentID": 2,
        "RegionalDivisionID": 4,
        "Type": 3
    },
    {
        "Code": null,
        "Name": "استان اصفهان",
        "ParentID": 1,
        "RegionalDivisionID": 5,
        "Type": 2
    },
    {
        "Code": null,
        "Name": "استان فارس",
        "ParentID": 1,
        "RegionalDivisionID": 6,
        "Type": 2
    },
    {
        "Code": "0711",
        "Name": "شیراز",
        "ParentID": 6,
        "RegionalDivisionID": 7,
        "Type": 3
    },
    {
        "Code": "0311",
 

In [7]:
# """


def salesStatementRegister(session, data):
    url = G_BaseURL + "/System/BusinessRuleEngine/Service.svc/call"
    headers = {"Cookie": session, "content-Type": "application/json"}
    JsonData = {"Name": "SalesStatementRegister", "Parameters": json.dumps(data)}
    response = requests.post(url, headers=headers, data=json.dumps(JsonData))
    if response.status_code != 200:
        raise Exception("salesStatementRegister".format(response.status_code))
    return response.json()


SalesStatement = {
    "Number": "",
    "CustomerRef": "6677",                           #شناسه مشتری

    "Date": "05/12/2024 00:00:00",
    "SalesOfficeRef": "1",                           #همیشه 1
    "SalesAreaRef": "1",                             #همیشه 1
    "RecipientType": "4",                            #نوع تحویل گیرنده
    "RecipientRef": "88",                          #شناسه مشتری تحویل گیرنده یا موسسه حمل
    "SellerBrokerRef": "1",                          #همیشه 1
    "CustomerBrokerRef": "1",                        #همیشه 1
    "DeliveryAddressRef": "26392",                   #شناسه آدرس مشتری
    "DeliveryAddressType": "1",                      # نوع نشانی همیشه مشتری  1 موسسه حمل 2
    "StatementType": "1",                            #همیشه 1 نوع اعلامیه
    "DeliverySpace": "08/09/2024 00:00:00",
    "ContractNo": str(random.randint(1, 99999999)),   #شماره قرارداد
    "PlantRef": "1",                                  #شماره مرکز همیشه 1
    "CalculationDate": "05/12/2024 16:59:29",
    "OperationalCurrencyExchangerAte" :"1",
    #"HiddenPayerAccountRef": "1482",    #شناسه حساب مشتری

    "ReferenceType": "1",                              #؟همیشه 1
    #"RecipientPartyRef": "9079",                       # شناسه شخص شرکت
    "SalesTypeRef": "1",                               #نوع فروش همیشه 1
    "InventoryRef": "3",                               #شناسه انبار همیشه 3
    "SalesStatementWageResults": [                     #کارمزدها
        {
            "BrokerServiceRef": "1",                   #کارمزد فروش بورسی
            "CalculationPrecedence": "0",              #همیشه 0
            "CurrencyRef": "1",                        #همیشه 1 نوع ارز
            "Value": "2502",                           #مبلغ کارمز
            "ValueSetByUser": True,                    #همیشه درست
        },
        {
            "BrokerServiceRef": "2",                   #کارمزد حق درج
            "CalculationPrecedence": "0",
            "CurrencyRef": "1",
            "Value": "300",
            "ValueSetByUser": True,
        },
    ],

    "SalesStatementItems": [                          #اقلام
        {
            "ProductRef": "9216",                      #شناسه محصول همیشه 9216؟
            "ProductUnitRef": "1",                     #واحد سنجش همیشه 1
            "InitialQuantity": "10",                   #مقدار
            "Fee": "900000",                           #نرخ

        }
    ],
}
# data = {"SalesStatement": SalesStatement}
#ERA = salesStatementRegister(session, {"SalesStatement": SalesStatement})
print(salesStatementRegister(session, {"SalesStatement": SalesStatement}))

{"SalesStatementID":0,"Errore":"0Exception has been thrown by the target of an invocation.System.Reflection.TargetInvocationException: Exception has been thrown by the target of an invocation. ---> SystemGroup.Framework.Exceptions.SgException: حساب انتخاب شده در سند متعلق به پرداخت کننده سند نمی باشد\r\n   at SystemGroup.Sales.SalesDocument.Business.SalesDocumentBusiness`4.Save(TSalesDocument& record, SalesDocumentSaveContext context) in D:\\agentB01\\_work\\118\\s\\GM\\Sales\\Imp\\SalesDocument\\Business\\SalesDocumentBusiness.cs:line 649\r\n   at SystemGroup.Sales.Brokerage.Business.SalesStatementBusiness.Save(SalesStatement& record, SalesDocumentSaveContext context) in D:\\agentB01\\_work\\118\\s\\GM\\Sales\\Imp\\Brokerage\\Business\\SalesStatement\\SalesStatementBusiness.cs:line 182\r\n   at Castle.Proxies.Invocations.SalesStatementBusiness_Save.InvokeMethodOnTarget()\r\n   at Castle.DynamicProxy.AbstractInvocation.Proceed()\r\n   at SystemGroup.Framework.Service.ServiceInterceptor